In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(survminer)
library(rlang)
library(cowplot)
library(patchwork)
library(ggpubr)

# Read in base data

In [ ]:
ready <- 
fread(paste0(SHARE_DIR, "biomarkers_base.csv")) %>% 
 fi(!is.na(purity), !is.na(durableClinicalBenefit)) %>% 
 mu(nrBor = abs(bestOverallResponse-1), non_response = abs(durableClinicalBenefit-1)) %>%
 se(-contains("geneset_mp_"))

In [ ]:
cohorts <- fread(paste0(SHARE_DIR, "fisher_base.csv")) %>% se(sampleId, cohortGo)

In [ ]:
highlights <- 
fread(paste0(SHARE_DIR,"share_with_fran_update.csv")) %>% 
 fi(!grepl("and_", feature)) %>% 
 se( cohortGo, feature) %>%
 rw() %>% 
 mu( feature_cont = strsplit(feature, "_gt")[[1]][1], 
     feature_cont = strsplit(feature_cont, "_lt")[[1]][1],
     feature_cont = strsplit(feature_cont, "_amp")[[1]][1]) %>% 
 ug() %>% 
 se(-feature) %>% 
 unique()

In [ ]:
#highlights

In [ ]:
features <- highlights %>% pu(feature_cont)

In [ ]:
s1 <- unlist(lapply(features, function(i) strsplit(i, "_gt")[[1]][1]))
s2 <- unlist(lapply(s1, function(i) strsplit(i, "_lt")[[1]][1]))
features_cont <- unlist(lapply(s2, function(i) strsplit(i, "_amp")[[1]][1])) %>% unique()

In [ ]:
go <-
ready %>% 
 se(sampleId, non_response, any_of(features_cont)) %>% 
 lj(cohorts, by = "sampleId") %>% 
 se(-sampleId) %>% 
 ga(feature_cont, val, -cohortGo, -non_response) %>%
 drop_na() %>% 
 ij(highlights, by = c("cohortGo", "feature_cont"))

- Combine

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 30)

In [ ]:
go %>% 
 ggplot( aes(x = as.factor(non_response), y = val, fill = as.factor(non_response))) + 
 go_theme + 
 geom_boxplot() + 
 facet_wrap(~cohortGo + feature_cont, scales = "free", ncol = 5)